# 02 — Trainer Arena (LoRA vs DoRA)

Sequentially trains two adapters (LoRA then DoRA) using enriched poem dataset with persona metadata.
Each run cleans up VRAM before the next begins.

**Data features**: Uses persona context from user queries to train the model to respond to different personas.

In [ ]:
# Cell 1: Imports
import gc
import json
from pathlib import Path
from IPython.display import clear_output
from typing import Dict, List, Any, Optional

import torch
from datasets import load_dataset, Dataset
from trl import SFTTrainer, SFTConfig
from unsloth import FastLanguageModel, is_bfloat16_supported


In [ ]:
# Cell 2: Config
project_root = Path('..').resolve()
data_path = project_root / 'data' / 'poem_finetune_13000.jsonl'  # Use enriched dataset with persona data
output_root = project_root / 'outputs'
base_model_id = 'unsloth/llama-3-8b-bnb-4bit'
max_seq_length = 4096
learning_rate = 2e-4
batch_size = 2
num_epochs = 2
gradient_accumulation = 4

configs = [
    {"name": "lora", "dora": False},
    {"name": "dora", "dora": True},
]

output_root.mkdir(parents=True, exist_ok=True)
print(f"✅ Config loaded. Data: {data_path.name}")

In [ ]:
# Cell 3: Load and transform enriched dataset
def load_enriched_dataset(path: str, max_samples: Optional[int] = None) -> List[Dict[str, str]]:
    """
    Transform enriched poem dataset into training format.
    
    Handles both:
    - Enriched records: user queries with persona metadata
    - Fallback records: plain string queries
    
    Returns list of {system, user, assistant} dicts.
    """
    training_examples = []
    stats = {"enriched": 0, "fallback": 0, "neutral": 0, "skipped": 0}
    
    system_prompt = (
        "You are a poetic wisdom keeper. When asked about the meaning or interpretation "
        "of a poetic line, respond with clarity that honors the original verse's depth. "
        "Provide thoughtful, contemplative responses that bridge classical language with "
        "modern understanding."
    )
    
    with open(path) as f:
        for line_no, line in enumerate(f, 1):
            if max_samples and len(training_examples) >= max_samples:
                break
            
            try:
                record = json.loads(line)
                
                # Extract core data
                poem_verse = record.get("poem_verse", "").strip()
                data = record.get("data", {})
                meaning = data.get("meaning", "").strip()
                queries = data.get("queries", {})
                
                if not poem_verse or not meaning:
                    stats["skipped"] += 1
                    continue
                
                # Process neutral queries (always plain strings)
                neutral_queries = queries.get("neutral", [])
                for query in neutral_queries:
                    if isinstance(query, str) and query.strip():
                        training_examples.append({
                            "system": system_prompt,
                            "user": query,
                            "assistant": meaning,
                        })
                        stats["neutral"] += 1
                
                # Process user queries (may have persona metadata)
                user_queries = queries.get("user", [])
                for query_item in user_queries:
                    if isinstance(query_item, dict):
                        # Enriched: has persona + query structure
                        persona = query_item.get("persona", {})
                        query = query_item.get("query", "").strip()
                        
                        if query:
                            # Build persona context prefix
                            persona_context = ""
                            if persona:
                                name = persona.get("name", "someone")
                                profession = persona.get("profession", "")
                                tone = persona.get("tone", "")
                                
                                parts = [f"[{name}"]
                                if profession:
                                    parts.append(f", {profession}")
                                if tone:
                                    parts.append(f", {tone}")
                                else:
                                    parts.append("]")
                                persona_context = "".join(parts) + ": "
                            
                            user_message = persona_context + query
                            training_examples.append({
                                "system": system_prompt,
                                "user": user_message,
                                "assistant": meaning,
                            })
                            stats["enriched"] += 1
                    
                    elif isinstance(query_item, str):
                        # Fallback: plain string query
                        if query_item.strip():
                            training_examples.append({
                                "system": system_prompt,
                                "user": query_item,
                                "assistant": meaning,
                            })
                            stats["fallback"] += 1
            
            except Exception as e:
                stats["skipped"] += 1
                if line_no <= 3:
                    print(f"⚠️  Line {line_no}: {type(e).__name__}: {str(e)[:60]}")
    
    print(f"\n📊 Dataset Transformation Summary:")
    print(f"   Neutral queries:            {stats['neutral']}")
    print(f"   User queries (enriched):    {stats['enriched']}")
    print(f"   User queries (fallback):    {stats['fallback']}")
    print(f"   Skipped:                    {stats['skipped']}")
    print(f"   ➜ Total examples:           {len(training_examples)}")
    
    return training_examples


# Load and transform the data
print("Loading enriched dataset...")
training_examples = load_enriched_dataset(str(data_path))

raw_ds = Dataset.from_dict({
    "system": [ex["system"] for ex in training_examples],
    "user": [ex["user"] for ex in training_examples],
    "assistant": [ex["assistant"] for ex in training_examples],
}).train_test_split(test_size=0.0)["train"]  # Keep all for training

print(f"\n✅ Dataset loaded: {len(raw_ds)} examples")
if raw_ds:
    print(f"\nSample training example:")
    sample = training_examples[0]
    print(f"  User:      {sample['user'][:70]}...")
    print(f"  Assistant: {sample['assistant'][:70]}...")

In [ ]:
# Cell 4: Training helper
def train_adapter(config):
    """
    Train a LoRA or DoRA adapter on the enriched poem dataset.
    """
    print(f"\n{'='*60}")
    print(f"🚀 Training {config['name'].upper()} adapter...")
    print(f"{'='*60}")
    
    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )

    def format_row(row):
        """
        Format a row into chat template.
        Works with pre-loaded system/user/assistant fields.
        """
        messages = [
            {"role": "system", "content": row["system"]},
            {"role": "user", "content": row["user"]},
            {"role": "assistant", "content": row["assistant"]},
        ]
        row["text"] = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )
        return row

    # Format dataset
    train_ds = raw_ds.map(
        format_row, 
        remove_columns=["system", "user", "assistant"],
        batched=False,
        desc="Formatting examples"
    )

    # Apply PEFT (LoRA or DoRA)
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        # lora_dropout=0.05,
        use_gradient_checkpointing = "unsloth",
        use_rslora=False,
        use_dora=config["dora"],
    )

    training_args = SFTConfig(
        output_dir=str(output_root / f"{config['name']}_runs"),
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation,
        weight_decay = 0.001,
        learning_rate=learning_rate,
        logging_steps=5,
        # bf16=is_bfloat16_supported(),
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        args=training_args,
    )

    print(f"Training on {len(train_ds)} examples...")
    trainer.train()

    adapter_dir = output_root / f"{config['name']}_adapter"
    adapter_dir.mkdir(parents=True, exist_ok=True)
    trainer.model.save_pretrained(adapter_dir)
    tokenizer.save_pretrained(adapter_dir)

    # Cleanup VRAM for next run
    del trainer, model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    clear_output(wait=True)
    print(f"Saved {config['name']} adapter to {adapter_dir}")


In [ ]:
# Cell 5: Run sequential A/B training
for cfg in configs:
    train_adapter(cfg)

print("\n" + "="*60)
print("✨ Training complete!")
print("="*60)
print(f"\nAdapters saved to: {output_root}")
print(f"  - {output_root / 'lora_adapter'}")
print(f"  - {output_root / 'dora_adapter'}")